In [1]:
import pandas as pd
#import from pypi
import tqdm
import torch
import warnings
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#local import
from CheXpert2.models.CNN import CNN
from CheXpert2.Metrics import Metrics
from CheXpert2.dataloaders.CXRLoader import CXRLoader
from CheXpert2.inference.inference import infer_loop,load_model
from CheXpert2 import names,hierarchy

for key in hierarchy.keys():
    if key not in names :
        names.insert(0,key)

In [10]:
@torch.no_grad()
def infer_loop(model, loader, criterion, device):
    """

    :param model: model to evaluate
    :param loader: dataset loader
    :param criterion: criterion to evaluate the loss
    :param device: device to do the computation on
    :return: val_loss for the N epoch, tensor of concatenated labels and predictions
    """
    running_loss = 0
    results = [torch.tensor([]), torch.tensor([])]

    for inputs, labels,idx in tqdm.tqdm(loader):
        # get the inputs; data is a list of [inputs, labels]

        inputs, labels = (
            inputs.to(device, non_blocking=True),
            labels.to(device, non_blocking=True),
        )
        #inputs,labels = loader.dataset.advanced_transform((inputs, labels))
        # forward + backward + optimize
        with torch.cuda.amp.autocast() :
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            outputs = torch.sigmoid(outputs)
            running_loss += loss.detach()

        if inputs.shape != labels.shape:  # prevent storing images if training unets
            results[1] = torch.cat(
                (results[1], outputs.detach().cpu()), dim=0
            )
            results[0] = torch.cat((results[0], labels.cpu()), dim=0)

        del (
            inputs,
            labels,
            outputs,
            loss,
        )  # garbage management sometimes fails with cuda


    return running_loss, results


In [0]:
criterion = torch.nn.BCEWithLogitsLoss()
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
    warnings.warn("No gpu is available for the computation")

# ----- parsing arguments -------------------------------------
#start = torch.cuda.Event(enable_timing=True)
#end = torch.cuda.Event(enable_timing=True)
# ------loading test set --------------------------------------
#img_dir = os.environ["img_dir"]
img_dir = "/mnt/e"
os.environ["DEBUG"]="True"
test_dataset = CXRLoader("Valid",img_dir, img_size=384,channels=3,datasets=["ChexPert"],prob=[0,]*5)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
)


In [11]:
# ----------------loading model -------------------------------
models = [
        CNN("convnext_base_384_in22ft1k", channels=3, num_classes=len(names), pretrained=False, hierarchical=True),

    ]
    # model =  torch.nn.parallel.DistributedDataParallel(model)

    # api = wandb.Api()
    # run = api.run(f"ccsmtl2/Chestxray/{args.run_id}")
    # run.file("models_weights/convnext_base/DistributedDataParallel.pt").download(replace=True)
weights = [
        "/mnt/c/Users/joeda/PycharmProjects/IA-MED_IMG/data/model_weights/atomic-wildflower.pt",

    ]


models=load_model(weights=weights,models=models)
i=0

CRITICAL:root:Server not available ; switching offline
/mnt/c/Users/joeda/PycharmProjects/IA-MED_IMG/CheXpert2/dataloaders/CXRLoader.py:184: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby("Exam ID").mean().round(0)


In [13]:

#start.record()
import time
start = time.time()
results = torch.zeros((len(test_dataset),len(names)))
for ex,model in enumerate(models) :
    model = model.to(device)
    running_loss, result = infer_loop(model=model, loader=test_loader, criterion=criterion, device=device)
    model = model.to("cpu")
    if ex==0 :
        results = result[1]
    else :
        results = torch.min(results,result[1])
#end.record()
end=time.time()
#torch.cuda.synchronize()
#print("time : ", start.elapsed_time(end))
print(end-start)
#plt.imshow(np.sum(heatmaps[0][0].detach().cpu().numpy(), axis=0))
#plt.savefig("heatmaps.png")

metric = Metrics(num_classes=len(names), names=names, threshold=np.zeros((len(names))) + 0.5)
metrics = metric.metrics()
metrics_results = {}
for key in metrics:
    pred = results.numpy()
    true = result[0].numpy().round(0)
    metric_result = metrics[key](true, pred)
    metrics_results[key] = metric_result

print(metrics_results)
i=0

100%|██████████| 100/100 [00:04<00:00, 21.92it/s]


19.300501823425293
(200, 10)
{'auc': {'Enlarged Cardiomediastinum': 0.6374436090225564, 'Cardiomegaly': 0.7668475802804161, 'Pleural Effusion': 0.9343979779411765, 'Pneumothorax': 0.7285223367697594, 'Lung Opacity': 0.9256978653530378, 'Atelectasis': 0.8353066666666668, 'Pneumonia': 0.8391927083333333, 'Consolidation': 0.8971354166666667, 'Edema': 0.9023508137432188, 'No Finding': 0.9447391688770999, 'mean': 0.8411634143653931}, 'f1': {'Enlarged Cardiomediastinum': 0.7218045112781954, 'Cardiomegaly': 0.5454545454545454, 'Pleural Effusion': 0.748091603053435, 'Pneumothorax': 0.37499999999999994, 'Lung Opacity': 0.875, 'Atelectasis': 0.6974358974358975, 'Pneumonia': 0.12698412698412698, 'Consolidation': 0.39751552795031053, 'Edema': 0.5714285714285715, 'No Finding': 0.9424657534246575, 'mean': 0.6001180537009738}, 'recall': {'Enlarged Cardiomediastinum': 0.9142857142857143, 'Cardiomegaly': 0.45454545454545453, 'Pleural Effusion': 0.765625, 'Pneumothorax': 0.5, 'Lung Opacity': 0.905172413

In [ ]:
"""
FOR A SPECIFIC IMAGE
"""
#inputs,labels=test_dataset[i]
#Manually loading images
import cv2 as cv
from pytorch_grad_cam import FullGrad
from PIL import Image
cam=FullGrad(model,target_layers=[])
img_size=384
img_dir=""
path0="c:/Users/joeda/PycharmProjects/IA-MED_IMG/data/1808916906062113_view0001.jpg"
path1="c:/Users/joeda/PycharmProjects/IA-MED_IMG/data/1808916906062113_view0002.jpg"
assert os.path.exists(path0)
image0 = np.array(cv.resize(cv.imread(f"{img_dir}{path0}", cv.IMREAD_GRAYSCALE),(img_size,img_size)))
image1 = np.array(cv.resize(cv.imread(f"{img_dir}{path1}", cv.IMREAD_GRAYSCALE),(img_size,img_size)))
inputs = np.concatenate([image0[None,:,:],image1[None,:,:]],axis=0)
inputs=inputs[None,:,:,:]
# inputs, labels = (
#             inputs.to(device, non_blocking=True),
#             labels.to(device, non_blocking=True),
#         )


inputs = torch.from_numpy(inputs).to(device,non_blocking=True)
# forward + backward + optimize
outputs = model(inputs)
img0=inputs[:,0:1,:,:]
heatmap0 = cam(img0.float()).squeeze() * -255+255

img1=inputs[:,1:2,:,:]
heatmap1 = cam(img1.float()).squeeze() * -255+255


heatmap0 = np.array(cv.applyColorMap(cv.cvtColor(heatmap0[:, :, None].astype(np.uint8), cv.COLOR_RGB2BGR),
                                          cv.COLORMAP_JET))

heatmap1 = np.array(cv.applyColorMap(cv.cvtColor(heatmap1[:, :, None].astype(np.uint8), cv.COLOR_RGB2BGR),
                                          cv.COLORMAP_JET))

Image.fromarray(heatmap0).save("c:/Users/joeda/PycharmProjects/IA-MED_IMG/dataheatmap_frontal.jpg")
Image.fromarray(heatmap1).save("c:/Users/joeda/PycharmProjects/IA-MED_IMG/dataheatmap_lateral.jpg")
img0=img0.cpu().numpy().astype(np.uint8).squeeze()
img1=img1.cpu().numpy().astype(np.uint8).squeeze()
img0 = cv.cvtColor(img0,cv.COLOR_GRAY2RGB)
img1=cv.cvtColor(img1,cv.COLOR_GRAY2RGB)
heatmap0 = cv.addWeighted(heatmap0, 0.5, img0, 0.5, 0)
heatmap1 = cv.addWeighted(heatmap1, 0.5, img1, 0.5, 0)
plt.imshow(heatmap0.squeeze())
plt.show()
plt.imshow(heatmap1.squeeze())
plt.show()
labels=torch.zeros_like(outputs).to(device)
#loss = criterion(outputs.squeeze(), labels)
outputs = torch.sigmoid(outputs)

i+=1
plt.imshow(inputs.squeeze().numpy())
data=pd.DataFrame([outputs.detach().numpy().squeeze(),labels.numpy().squeeze()],columns=names,index=["preds","ground-truth"])
print(data.T)


In [ ]:

#inputs,labels=test_dataset[i]
#Manually loading images
import cv2 as cv
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam import FullGrad,EigenCAM,GradCAM,HiResCAM,GradCAMPlusPlus,ScoreCAM
from PIL import Image

model = models[0] # only using 1 model, not an Ensemble
model = model.to(device)
cams=[
    #FullGrad(model,target_layers=[]), #crashes
    #EigenCAM(model,target_layers=[model.backbone.stages[-1 , #same as gradcam
    GradCAM(model,target_layers=[model.backbone.stages[-1]]),
    HiResCAM(model,target_layers=[model.backbone.stages[-1]]),
    ScoreCAM(model,target_layers=[model.backbone.stages[-1]],use_cuda=True),
]
setattr(cams[-1],"batch_size",16)
img_size=384

inputs,labels,idx = test_dataset[i]
inputs = inputs[None,:,:,:]
inputs = inputs.to(device,non_blocking=True)
# forward + backward + optimize
outputs = model(inputs)
model.hierarchical=False
channels=3
i=0
img0=inputs[0:1,i*channels : (i+1)*channels,:,:]
i=1
img1=inputs[0:1,i*channels : (i+1)*channels,:,:]
# create figure
fig = plt.figure(figsize=(10, 7))

img = test_dataset.read_img(i)
img0_display=img[:,:,0].astype(np.uint8)
i=1
img1_display=img[:,:,3].astype(np.uint8)
if channels ==1 :
    img0_display = cv.cvtColor(img0_display,cv.COLOR_GRAY2RGB)
    img1_display=cv.cvtColor(img1_display,cv.COLOR_GRAY2RGB)

rows=2
columns=len(cams)

targets = [ClassifierOutputTarget(i) for i,output in enumerate(outputs.squeeze().cpu().tolist()) if output>0.5]

for ex,cam in enumerate(cams,start=1) :
    heatmap0 = cam(img0.float(),targets=targets).squeeze() * -255+255
   
    heatmap1 = cam(img1.float(),targets=targets).squeeze() * -255+255
   

    heatmap0 = np.where(heatmap0<0.9,0,heatmap0)
    heatmap1 = np.where(heatmap0<0.9,0,heatmap1)
    fig.add_subplot(rows, columns, ex)
    heatmap0 = np.array(cv.applyColorMap(cv.cvtColor(heatmap0[:, :,None].astype(np.uint8), cv.COLOR_RGB2BGR),cv.COLORMAP_JET))

    heatmap0 = cv.addWeighted(heatmap0, 0.5, img0_display, 0.5, 0)
    plt.imshow(heatmap0.squeeze())

    fig.add_subplot(rows, columns, len(cams)+ex)
    heatmap1 = np.array(cv.applyColorMap(cv.cvtColor(heatmap1[:, :,None].astype(np.uint8), cv.COLOR_RGB2BGR),cv.COLORMAP_JET))
    heatmap1 = cv.addWeighted(heatmap1, 0.5, img1_display, 0.5, 0)
    plt.imshow(heatmap1.squeeze())

i+=1
plt.show()
plt.imshow(img0_display)
plt.show()
plt.imshow(img1_display)
plt.show()
#Image.fromarray(heatmap0).save("c:/Users/joeda/PycharmProjects/IA-MED_IMG/dataheatmap_frontal.jpg")
#Image.fromarray(heatmap1).save("c:/Users/joeda/PycharmProjects/IA-MED_IMG/dataheatmap_lateral.jpg")

labels=torch.zeros_like(outputs).to(device)
#loss = criterion(outputs.squeeze(), labels)
outputs = torch.sigmoid(outputs)

data=pd.DataFrame([outputs.cpu().detach().numpy().squeeze(),labels.cpu().numpy().squeeze()],columns=names,index=["preds","ground-truth"])

print(data.T)

